In [1]:
def hello(thing):
    return "Hello " + thing


In [2]:
hello("python WA")


'Hello python WA'

In [3]:
hello(3)


TypeError: can only concatenate str (not "int") to str

In [4]:
def hello(thing: str) -> str:
    return "Hello " + thing


In [5]:
hello(33)


TypeError: can only concatenate str (not "int") to str

In [9]:
import datetime 


def day_of_week(lookup_date: datetime.date) -> str:
    return lookup_date.weekday()


In [10]:

day_of_week("2000-01-01")


AttributeError: 'str' object has no attribute 'weekday'

In [11]:
day_of_week(datetime.date(2000,1,1))


5

In [16]:
def day_of_week(lookup_date: datetime.date) -> str:
    return lookup_date.strftime('%A')


In [41]:
day_of_week(datetime.date(2000,1,1))

'Saturday'

## classes

In [32]:

class Person:
    def __init__(self, name: str, birthday: datetime.date, gender: str) -> None:
        self.name : str = name
        self.birthday: datetime.date = birthday
        self.gender: str = gender
        
    def day_of_birth(self) -> str:
        return day_of_week(self.birthday)
    
    def __repr__(self) -> str:
        return f"{self.name} ({self.birthday}) {self.gender}"

In [33]:
jane = Person("Jane", datetime.date(2000,1,1))

TypeError: Person.__init__() missing 1 required positional argument: 'gender'

In [35]:
jane: Person = Person("Jane", datetime.date(2000,1,1), "female")
jane


Jane (2000-01-01) female

In [36]:
class Person:
    def __init__(self, name: str, birthday: datetime.date, gender: str | None = None) -> None:
        self.name : str = name
        self.birthday: datetime.date = birthday
        self.gender: str | None = gender
        
    def day_of_birth(self) -> str:
        return day_of_week(self.birthday)
    
    def __repr__(self) -> str:
        return f"{self.name} ({self.birthday}) {self.gender}"

In [38]:
jane: Person = Person("Jane", datetime.date(2000,1,1))
jane


Jane (2000-01-01) None

## typed dict


## data classes
